### Graph based Music Recommender. Task 4

#### Task description
Data description (DataFrames in parquet format)
Location - /data/sample264

Fields: trackId, userId, timestamp, artistId

trackId - id of the track
userId - id of the user
artistId - id of the artist
timestamp - timestamp of the moment the user starts listening to a track
Location - /data/meta

Fields: type, Name, Artist, Id

Type could be “track” or “artist”
Name is the title of the track if the type == “track” and the name of the musician or group if the type == “artist”.
Artist states for the creator of the track in case the type == “track” and for the name of the musician or group in case the type == “artist”.
Id - id of the item
Task
Build the edges of the type “artist-track”. Take the amount of times the track HAS BEEN listened by all users as the weight of the edge from the artist’s vertex to the track’s vertex. For each artist take top-100 tracks and normalize weights.

Sort the resulting Data Frame in descending order by the column norm_count, take top 40 rows, select only the columns “id1”, “id2”, sort them in ascending order this time first by “id1”, then by “id2” and print the columns “id1”, “id2” of the resulting dataframe.

In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [3]:
window1 = Window.partitionBy('artistId').orderBy(col('count').desc())
window2 = Window.partitionBy('artistId')

userToArtist = data.groupBy(col("artistId"), col("trackId")) \
                   .count() \
                   .withColumn("rank", rank().over(window1)) \
                   .filter(col("rank") <= 100) \
                   .withColumn('sum_count', sum(col('count')).over(window2)) \
                   .withColumn('norm_count', col('count') / col('sum_count') * 0.5) \
                   .orderBy(desc("norm_count"), asc("artistId"), asc("trackId")) \
                   .limit(40) \
                   .select(col("artistId"), col("trackId")) \
                   .collect()

In [4]:
for val in userToArtist:
    print "%s %s" % val

967993 869415
967998 947428
968004 927380
968017 859321
968022 852786
968034 807671
968038 964150
968042 835935
968043 913568
968046 935077
968047 806127
968065 907906
968073 964586
968086 813446
968092 837129
968118 914441
968125 821410
968140 953008
968148 877445
968161 809793
968163 803065
968168 876119
968189 858639
968221 896937
968224 892880
968232 825536
968237 932845
968238 939177
968241 879045
968242 911250
968248 953554
968255 808494
968259 880230
968265 950148
968266 824437
968269 913243
968272 816049
968278 946743
968285 847460
968286 940006
